In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [36]:
df = pd.read_csv('../../data_cleaned/merged/all_data_2018_2025_dayahead.csv')
df.head()

,price,c_by_hour,load_forecast_da,off_wind_da,on_wind_da,solar_da,gen_forecast_da,res_sum_da,date,year,month,day,weekday,hour,dayofyear,is_holiday,day_type
0,28.32,4,43647.5450,5068.9000,18879.7900,0.0,51070.88,23948.6900,2019-01-01,2019,1,1,1,0,1,1,holiday
1,10.07,4,41692.5650,5042.4425,20626.5625,0.0,51084.26,25669.0050,2019-01-01,2019,1,1,1,1,1,1,holiday
2,-4.08,4,40587.2325,5028.4175,22355.6850,0.0,51512.67,27384.1025,2019-01-01,2019,1,1,1,2,1,1,holiday
3,-9.91,4,40308.2500,4977.6500,24032.4775,0.0,52693.45,29010.1275,2019-01-01,2019,1,1,1,3,1,1,holiday
4,-7.41,4,40659.5750,4907.4000,25452.1675,0.0,53666.46,30359.5675,2019-01-01,2019,1,1,1,4,1,1,holiday


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17544 entries, 0 to 17543
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   price             17544 non-null  float64       
 1   c_by_hour         17544 non-null  int64         
 2   load_forecast_da  17544 non-null  float64       
 3   off_wind_da       17544 non-null  float64       
 4   on_wind_da        17544 non-null  float64       
 5   solar_da          17544 non-null  float64       
 6   gen_forecast_da   17544 non-null  float64       
 7   res_sum_da        17544 non-null  float64       
 8   date              17544 non-null  datetime64[ns]
 9   year              17544 non-null  int64         
 10  month             17544 non-null  int64         
 11  day               17544 non-null  int64         
 12  weekday           17544 non-null  int64         
 13  hour              17544 non-null  int64         
 14  dayofyear         17544 non

In [38]:
df = df.query('year < 2021')

In [43]:
df['date']= pd.to_datetime(df['date'])

In [53]:
df['season_hour']= df.hour.map(df.groupby('hour')['price'].mean())

In [54]:
df['season_weekday']=df.weekday.map(df.groupby('weekday')['price'].mean())

In [56]:
df['season_year']=df.dayofyear.map(df.groupby('dayofyear')['price'].mean().rolling(7,center=True).mean())

In [58]:
df['residual']= (df.price - df.season_hour - df.season_weekday - df.season_year)

In [63]:
p_value = adfuller(df['residual'].dropna())[1]
p_value

1.5906492375884469e-21

In [64]:
df['residual_diff']= df['residual'].diff()

In [67]:
arma_model = ARIMA(df['residual'].dropna(), order=(1,0,1))
arma_res = arma_model.fit()
print(arma_res.summary())

/home/saadu/bootcamp_ds_ai/capstone_project/ai-for-energy/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/saadu/bootcamp_ds_ai/capstone_project/ai-for-energy/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/saadu/bootcamp_ds_ai/capstone_project/ai-for-energy/.venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:               residual   No. Observations:                17280
Model:                 ARIMA(1, 0, 1)   Log Likelihood              -50313.251
Date:                Tue, 10 Feb 2026   AIC                         100634.503
Time:                        22:54:14   BIC                         100665.532
Sample:                             0   HQIC                        100644.727
                              - 17280                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -68.4904      0.476   -144.018      0.000     -69.422     -67.558
ar.L1          0.9102      0.002    587.735      0.000       0.907       0.913
ma.L1          0.2239      0.003     85.811      0.0